In [1]:
import cv2
import os
import numpy as np
np.random.seed(1120)
from sklearn.model_selection import train_test_split

top_path = 'C:/Users/USER/Desktop/mask_data/' # 안에 chin_mask , full_mask... 로 하위 폴더가 있는 상위 폴더 경로

save_path = 'C:/Users/USER/Desktop/save_data/' # 저장할 경로 (안에 빈 train , test 폴더 먼저 만들어 놓고 실행)

folders = os.listdir(top_path)

for folder in folders:
    
    files = os.listdir(top_path+folder+'/')
    
    image_files = [file for file in files if file.endswith(".png")]
    
    image_1000 = image_files[-1000:]
    
    train,test = train_test_split(image_1000,test_size=0.3)
    
    for img in train:
        original=cv2.imread(top_path+folder+'/'+img)
        cv2.imwrite(save_path+'train/'+folder+'/'+img, original.copy())
        
    for img in test:
        original=cv2.imread(top_path+folder+'/'+img)
        cv2.imwrite(save_path+'test/'+folder+'/'+img, original.copy())

In [2]:
# IMAGE_WIDTH=128
# IMAGE_HEIGHT=128
# IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
# IMAGE_CHANNELS=3

In [23]:
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout
from keras.optimizers import Adam

In [41]:
import pandas as pd

In [53]:
filenames=os.listdir("C:/Users/USER/Desktop/save_data/train")
categories = []
for filename in filenames:
    category = filename.split(',')[0]
    # print(category)
    if category == 'chin_mask':
        categories.append(0)
    elif category == 'full_mask':
        categories.append(1)
    elif category == 'nose_mask':
        categories.append(2)
    elif category == 'no_mask':
        categories.append(3)

In [54]:
df=pd.DataFrame({
    'filename': filenames,
    'category' : categories
})

In [55]:
df

,filename,category
0,chin_mask,0
1,full_mask,1
2,nose_mask,2
3,no_mask,3


In [56]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(120, 90, 3)))
model.add(MaxPool2D(strides=2))
model.add(Conv2D(filters=48, kernel_size=(5,5), padding='valid', activation='relu'))
model.add(MaxPool2D(strides=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(84, activation='relu'))
model.add(Dense(4, activation='softmax'))

model.build()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 120, 90, 32)       2432      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 60, 45, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 56, 41, 48)        38448     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 20, 48)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 26880)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 256)               6881536   
_________________________________________________________________
dense_9 (Dense)              (None, 84)               

In [57]:
adam = Adam(lr=5e-4)
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

In [27]:
# from keras.callbacks import ReduceLROnPlateau

In [58]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [59]:
earlystop = EarlyStopping(patience=10)

In [60]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                patience=3, 
                                verbose=1, 
                                factor=0.2, 
                                min_lr=1e-6)

In [61]:
callbacks = [earlystop, learning_rate_reduction]

In [66]:
train_datagen = ImageDataGenerator(rescale = 1/255.,
                                  rotation_range=15,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  shear_range=0.5,
                                  zoom_range=[0.8,2.0],
                                  horizontal_flip=True,
                                  vertical_flip=True,
                                  fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1/255.)

In [67]:
train_generator=train_datagen.flow_from_directory(
    "C:/Users/USER/Desktop/save_data/train/",
    batch_size=10,
    target_size=(120,90),
    class_mode='categorical')

validation_generator=test_datagen.flow_from_directory(
    "C:/Users/USER/Desktop/save_data/test/",
    batch_size=10,
    target_size=(120,90),
    class_mode='categorical')

Found 2800 images belonging to 4 classes.
Found 1200 images belonging to 4 classes.


In [68]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    steps_per_epoch=280,
    epochs=50,
    validation_steps=120
)

Epoch 1/50
280/280 [==============================] - 78s 278ms/step - loss: 0.5785 - accuracy: 0.7650 - val_loss: 0.4530 - val_accuracy: 0.8217
Epoch 2/50
280/280 [==============================] - 66s 235ms/step - loss: 0.3343 - accuracy: 0.8775 - val_loss: 0.3229 - val_accuracy: 0.9067
Epoch 3/50
280/280 [==============================] - 67s 239ms/step - loss: 0.2705 - accuracy: 0.9089 - val_loss: 0.1244 - val_accuracy: 0.9733
Epoch 4/50
280/280 [==============================] - 79s 282ms/step - loss: 0.2223 - accuracy: 0.9243 - val_loss: 0.1182 - val_accuracy: 0.9667
Epoch 5/50
280/280 [==============================] - 85s 305ms/step - loss: 0.2025 - accuracy: 0.9293 - val_loss: 0.1220 - val_accuracy: 0.9600
Epoch 6/50
280/280 [==============================] - 93s 333ms/step - loss: 0.1919 - accuracy: 0.9375 - val_loss: 0.0946 - val_accuracy: 0.9767
Epoch 7/50
280/280 [==============================] - 87s 310ms/step - loss: 0.1990 - accuracy: 0.9336 - val_loss: 0.0953 - val_ac

In [69]:
from keras.models import load_model
model.save('C:/Users/USER/Desktop/from_LenetV5.h5')

